# 🛒 Amazon Product Dataset: Data Cleaning & Preprocessing
**Project:** Amazon Sales Analysis  
**Team:** [Elena Moysidou, Antonis Karousis, Christos Karlaftis]

---

### 📝 Project Overview
This notebook focuses on **cleaning and preprocessing** the Amazon Products dataset. The goal is to transform the raw data into a structured format suitable for analysis and visualization.

### 🔧 Key Steps:
1. Handling **missing values** and data inconsistencies.
2. Converting **currency columns** (Rupees to Euros).
3. Cleaning **string columns** (Ratings, Percentages).
4. Feature Engineering (Creating new categories).

In [1]:
import pandas as pd
import numpy as np

# Rupee-Euro exchange rate constant
INR_TO_EUR_RATE = 0.011

def clean_amazon_data(df):
    """
    Accepts the raw Amazon DataFrame and returns the cleaned dataset.
    """
    # 1. Select relevant columns.
    selected_columns = [
        'product_id', 'product_name', 'category', 'discounted_price',
        'actual_price', 'discount_percentage', 'rating', 'rating_count',
        'about_product', 'user_id', 'user_name', 'product_link'
    ]
    df_clean = df[selected_columns].copy()

    # 2. Drop rows with missing 'rating_count'.
    df_clean = df_clean.dropna(subset=['rating_count'])

    # 3. Clean 'discount_percentage': remove '%' and cast to float.
    df_clean['discount_percentage'] = (
        df_clean['discount_percentage']
        .astype(str)
        .str.replace("%", "", regex=False)
        .astype(float)
    )

    # 4. Convert 'rating' to numeric, coercing errors to NaN.
    df_clean['rating'] = pd.to_numeric(df_clean['rating'], errors='coerce')
    
    # Drop rows with NaN values in 'rating' resulting from coercion.
    df_clean = df_clean.dropna(subset=['rating'])

    # 5. Clean 'rating_count': remove commas and cast to float.
    df_clean['rating_count'] = (
        df_clean['rating_count']
        .astype(str)
        .str.replace(",", "", regex=False)
        .astype(float)
    )

    # 6. Extract 'main_category' from 'category' string.
    df_clean['main_category'] = df_clean['category'].str.split('|').str[0]

    # 7. Helper function for price cleaning.
    def clean_currency_column(series):
        return (
            series
            .astype(str)
            .str.replace('₹', '', regex=False)
            .str.replace(',', '', regex=False)
            .astype(float)
        )

    # Apply cleaning function to price columns.
    df_clean['discounted_price_float_indian'] = clean_currency_column(df_clean['discounted_price'])
    df_clean['actual_price_float_indian'] = clean_currency_column(df_clean['actual_price'])

    # 8. Convert prices to Euros.
    df_clean['discounted_price_euros'] = df_clean['discounted_price_float_indian'] * INR_TO_EUR_RATE
    df_clean['actual_price_euros'] = df_clean['actual_price_float_indian'] * INR_TO_EUR_RATE

    # Round prices to 2 decimal places.
    df_clean['discounted_price_euros'] = df_clean['discounted_price_euros'].round(2)
    df_clean['actual_price_euros'] = df_clean['actual_price_euros'].round(2)

    return df_clean

# --- Main Execution ---
if __name__ == "__main__":
    # Load the dataset.
    try:
        print("Load the dataset...")
        df = pd.read_csv('amazon.csv')
        
        # Cleaning
        print("Cleaning dataset...")
        cleaned_df = clean_amazon_data(df)
        
        # Display cleaned data info.
        print("\n--- Cleaned Data Info ---")
        cleaned_df.info()
        print("\nFirst 5 rows:")
        print(cleaned_df.head())

        # Save data (optional)
        # cleaned_df.to_csv('amazon_cleaned.csv', index=False)
        # print("File saved as 'amazon_cleaned.csv'")

    except FileNotFoundError:
        print("Error: File 'amazon.csv' not found.")

Load the dataset...
Cleaning dataset...

--- Cleaned Data Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 1462 entries, 0 to 1464
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     1462 non-null   object 
 1   product_name                   1462 non-null   object 
 2   category                       1462 non-null   object 
 3   discounted_price               1462 non-null   object 
 4   actual_price                   1462 non-null   object 
 5   discount_percentage            1462 non-null   float64
 6   rating                         1462 non-null   float64
 7   rating_count                   1462 non-null   float64
 8   about_product                  1462 non-null   object 
 9   user_id                        1462 non-null   object 
 10  user_name                      1462 non-null   object 
 11  product_link                   1462 non-null  